In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
from pathlib import Path
root_path = Path.cwd().parent.absolute()
import sys
sys.path.append(str(root_path))
import os

In [2]:
from architectures.MMNN import SinTUActivation,MMNN
from architectures.node import Parametric_NODE,init_node
from solvers import string_2_solver

In [3]:
from typing import Sequence, Callable, Optional, Dict, Any
from typing import List
from functools import partial

import jax
import jax.numpy as jnp
import jax.random as jrandom
import flax.linen as nn
import optax
import diffrax

import matplotlib.pyplot as plt
# plt.style.use('figures/images_style.mplstyle')
import numpy as np

In [4]:
device = jax.devices()[0] if jax.devices() else jax.devices('cpu')[0]

In [5]:
# Create model 

d_space = 2
include_time = True
if include_time:
    ranks = [d_space+1]+ [18]*5 + [ 2]     # Adjusted ranks for time
else:
    ranks = [2]+ [18]*5 + [ 2]     # Adjusted ranks for no time
widths = [356]*6      #


mmnn = MMNN(
    ranks=ranks,
    widths=widths,
    activation=SinTUActivation(),
    ResNet=False,
    FixWb=True
)

parameters = init_node(mmnn, d_space=d_space, include_time=include_time)


node = Parametric_NODE(
    model = mmnn,
    parameters=parameters,
    include_time=include_time,
    solver='euler',
    dt=0.01,
    history=False
)



In [6]:
t_ = jnp.linspace(0, 1, 100)
y0 = jnp.array([[1.0, 0.0]])

y1 = node.forward(
    parameters=parameters,
    t_list=t_,
    y0=y0
)